# Influence Spacer 3

In [1]:
## Imports:

import sys
import glob

import numpy as np
from numpy import exp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
from matplotlib.image import NonUniformImage
from matplotlib.patches import Rectangle
%matplotlib inline

import pandas as pd
import math

from scipy import optimize
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy import stats
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.interpolate import spline
from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import griddata
from scipy.stats import chisquare
from scipy.stats import power_divergence

## General Definitions and Parameters

In [2]:
## General function definitions:

def f_polar_R(cartesian_x, cartesian_y):
    return np.sqrt(cartesian_x * cartesian_x + cartesian_y * cartesian_y)

def f_polar_Phi(cartesian_x, cartesian_y):
    return np.arctan2(cartesian_y, cartesian_x)

def f_distance(x_a, y_a, x_b, y_b):
    return np.sqrt((x_b - x_a)**2 + (y_b - y_a)**2)


def flatten_tuples(tup):
    return np.array([element for tupl in tup for element in tupl])

def sum_tuple_subentries(tup):
    return [np.sum(tupl) for tupl in tup]

def nth_tuple_subentries(tup, n):
    return [tupl[n] for tupl in tup]


def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

In [3]:
## PMT properties
R8520_body_width = 25.7
R8520_body_cornerradius = 2.
R8520_photocathode_width = 20.5
R8520_photocathode_cornerradius = 1.
PMT_ID = np.array([1,2,3,4,5,6,7])
PMT_position_x = np.array([-14.,-28,-14.,14.,28.,14.,0.])      # x-position PMTs in mm in cartesian coordinates
PMT_position_y = np.array([-28.,0.,28.,28.,0.,-28.,0.])        # y-position PMTs in mm in cartesian coordinates
PMT_position_R = f_polar_R(PMT_position_x, PMT_position_y)     # radial position PMTs in mm in polar coordinates
PMT_position_Phi = f_polar_Phi(PMT_position_x, PMT_position_y) # angular position PMTs in rad in polar coordinates
PMT_positions = pd.DataFrame(index=PMT_ID, data={'PMT_ID': PMT_ID, 'PMT_position_x': PMT_position_x, 'PMT_position_y': PMT_position_y, 'PMT_position_R': PMT_position_R, 'PMT_position_Phi': PMT_position_Phi})

## Imports and Data Processing

In [18]:
!ls /media/alex/TOSHIBA\ EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/*_ReconstructedPosition_Deviations_Centroids.pkl
!ls /media/alex/TOSHIBA\ EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/*_Goodness_fiducialisation_Centroids.pkl
!ls /media/ab602/TOSHIBA\ EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/*_ReconstructedPosition_Deviations_Centroids.pkl
!ls /media/ab602/TOSHIBA\ EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/*_Goodness_fiducialisation_Centroids.pkl

'/media/alex/TOSHIBA EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/optPhot_S2_1.0e+06_pmtGap0.0_RPTFE0.95_LXeAbs100_Tmesh0.89770509_nop_1.0e+04_ReconstructedPosition_Deviations_Centroids.pkl'
'/media/alex/TOSHIBA EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/optPhot_S2_1.0e+06_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_ReconstructedPosition_Deviations_Centroids.pkl'
'/media/alex/TOSHIBA EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/optPhot_S2_1.0e+07_pmtGap0_RPTFE0.95_LXeAbs100_Tmesh0.89770509_ReconstructedPosition_Deviations_Centroids.pkl'
'/media/alex/TOSHIBA EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/optPhot_S2_1.0e+07_pmtGap10c_RPTFE0.95_LXeAbs100_Tmesh0.89770509_ReconstructedPosition_Deviations_Centroids.pkl'
'/media/alex/TOSHIBA EXT/Thesis/Simulation_Outputs/Position_Reconstruction_LRF/optPhot_S2_1.0e+07_pmtGap13-23_RPTFE0.95_LXeAbs100_Tmesh0.89770509_ReconstructedPosition_Deviations_Centroids.pkl'
